# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [ ]:
# try this project here!

In [1]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)
from torch import nn, optim

W0721 21:22:40.852053 140275573384960 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/raj/mooc/private-ai/.venv/lib/python3.6/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0721 21:22:40.862836 140275573384960 deprecation_wrapper.py:119] From /home/raj/mooc/private-ai/.venv/lib/python3.6/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
# create workers

bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')
secure_worker = sy.VirtualWorker(hook, id='secure_worker')

In [ ]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

In [ ]:
data = th.randint(0,2, (10,10)).type(th.FloatTensor)
target = th.randint(0,2,(10,1)).type(th.FloatTensor)

In [ ]:
bobs_data = data[:5].send(bob)
bobs_target = target[:5].send(bob)

In [ ]:
alice_data = data[5:].send(alice)
alice_target = target[5:].send(alice)

In [ ]:
model = nn.Linear(10,1)

In [ ]:
epoch = 10

for round_iter in range(130):
    bobs_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)

    bobs_optim = optim.SGD(params=bobs_model.parameters(), lr=0.01)
    alice_optim = optim.SGD(params=alice_model.parameters(), lr=0.01)

    for i in range(epoch):
        bobs_optim.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_target-bobs_pred)**2).sum()
        bobs_loss.backward()
        bobs_optim.step()
        bobs_loss = bobs_loss.get()

        alice_optim.zero_grad()
        alice_pred = alice_model(alice_data)
        alice_loss = ((alice_target-alice_pred)**2).sum()
        alice_loss.backward()
        alice_optim.step()
        alice_loss = alice_loss.get()
#         print("Bobs loss:",bobs_loss.data, "Alice loss:", alice_loss.data)
        


    alice_model.move(secure_worker)
    bobs_model.move(secure_worker)
    
    with th.no_grad():
        model.weight.set_(((alice_model.weight.data + bobs_model.weight.data)/2).get())
        model.bias.set_(((alice_model.bias.data + bobs_model.bias.data)/2).get())
    
    print("Bobs loss:",bobs_loss.data, "Alice loss:", alice_loss.data)
    

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [ ]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [ ]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [ ]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

This even works for addition between two shared values!!

In [ ]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [ ]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

In [ ]:
(bob_x_share + alice_x_share) % Q

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [ ]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [ ]:
# try this project here!

In [ ]:
import random
Q = 23740629843760239486723
x = 5

def encrypt(x, n_shares=3):
    x_share = list()
    for i in range(n_shares-1):
        x_share.append(random.randint(0,Q))
    
    x_share.append(Q-(sum(x_share)%Q)+x)
    
    return tuple(x_share)

In [ ]:
def decrypt(shares):
    return sum(shares)%Q

In [ ]:
def add (a,b):
    c = list()
    assert len(a)==len(b)
    
    for i in range(len(a)):
        c.append((a[i]+b[i])%Q)
    return tuple(c)

In [ ]:
share_1 = encrypt(5,10)
share_2 = encrypt(7,10)

share_added = add(share_1, share_2)

decrypt_added_value = decrypt(share_added)

print(decrypt_added_value)

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [ ]:
BASE=10
PRECISION=4

In [ ]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [ ]:
encode(3.5)

In [ ]:
decode(35000)

In [ ]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [ ]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [ ]:
x = th.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [ ]:
x = x.share(bob, alice, secure_worker)

In [ ]:
bob._objects

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [ ]:
y = x + x

In [ ]:
y

In [ ]:
y.get()

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [ ]:
x = th.tensor([0.1,0.2,0.3])

In [ ]:
x

In [ ]:
x = x.fix_prec()

In [ ]:
x.child.child

In [ ]:
y = x + x

In [ ]:
y = y.float_prec()
y

### Shared Fixed Precision

And of course, we can combine the two!

In [34]:
x = th.tensor([0.1, 0.2, 0.3])

In [35]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [36]:
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:47727338810 -> bob:92254959658]
	-> (Wrapper)>[PointerTensor | me:86204787627 -> alice:98451413291]
	-> (Wrapper)>[PointerTensor | me:15632897329 -> secure_worker:67257360172]
	*crypto provider: me*

In [37]:
y = x + x

In [38]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation

In [3]:
alice.clear_objects()
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [4]:
data = th.randint(0,2, (10,10)).type(th.FloatTensor)
target = th.randint(0,2,(10,1)).type(th.FloatTensor)

In [5]:
bobs_data = data[:5].send(bob)
bobs_target = target[:5].send(bob)

In [6]:
alice_data = data[5:].send(alice)
alice_target = target[5:].send(alice)

In [7]:
model = nn.Linear(10,1)

In [ ]:
epoch = 10

for round_iter in range(10000):
    bobs_model = model.copy().send(bob)
    alice_model = model.copy().send(alice)

    bobs_optim = optim.SGD(params=bobs_model.parameters(), lr=0.01)
    alice_optim = optim.SGD(params=alice_model.parameters(), lr=0.01)

    for i in range(epoch):
        bobs_optim.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_target-bobs_pred)**2).sum()
        bobs_loss.backward()
        bobs_optim.step()
        bobs_loss = bobs_loss.get()

        alice_optim.zero_grad()
        alice_pred = alice_model(alice_data)
        alice_loss = ((alice_target-alice_pred)**2).sum()
        alice_loss.backward()
        alice_optim.step()
        alice_loss = alice_loss.get()
#         print("Bobs loss:",bobs_loss.data, "Alice loss:", alice_loss.data)
        
    alice_model_weights = alice_model.weight.data.clone().get().fix_prec().share(bob,alice)
    bobs_model_weights = bobs_model.weight.data.clone().get().fix_prec().share(bob, alice)
    
    alice_model_bias = alice_model.bias.data.clone().get().fix_prec().share(bob, alice)
    bobs_model_bias = bobs_model.bias.data.clone().get().fix_prec().share(bob, alice)
    
    with th.no_grad():
        model.weight.set_(((alice_model_weights + bobs_model_weights)/2).get().float_prec())
        model.bias.set_(((alice_model_bias+ bobs_model_bias)/2).get().float_prec())
    
    print("Bobs loss:",bobs_loss.data, "Alice loss:", alice_loss.data)
    

Bobs loss: tensor(0.0267) Alice loss: tensor(0.1039)
Bobs loss: tensor(0.0262) Alice loss: tensor(0.1026)
Bobs loss: tensor(0.0258) Alice loss: tensor(0.1014)
Bobs loss: tensor(0.0254) Alice loss: tensor(0.1002)
Bobs loss: tensor(0.0250) Alice loss: tensor(0.0992)
Bobs loss: tensor(0.0247) Alice loss: tensor(0.0981)
Bobs loss: tensor(0.0244) Alice loss: tensor(0.0969)
Bobs loss: tensor(0.0240) Alice loss: tensor(0.0958)
Bobs loss: tensor(0.0235) Alice loss: tensor(0.0950)
Bobs loss: tensor(0.0231) Alice loss: tensor(0.0939)
Bobs loss: tensor(0.0229) Alice loss: tensor(0.0927)
Bobs loss: tensor(0.0227) Alice loss: tensor(0.0916)
Bobs loss: tensor(0.0223) Alice loss: tensor(0.0907)
Bobs loss: tensor(0.0219) Alice loss: tensor(0.0896)
Bobs loss: tensor(0.0215) Alice loss: tensor(0.0889)
Bobs loss: tensor(0.0211) Alice loss: tensor(0.0880)
Bobs loss: tensor(0.0209) Alice loss: tensor(0.0869)
Bobs loss: tensor(0.0207) Alice loss: tensor(0.0859)
Bobs loss: tensor(0.0203) Alice loss: tensor(0

Bobs loss: tensor(0.0045) Alice loss: tensor(0.0285)
Bobs loss: tensor(0.0045) Alice loss: tensor(0.0283)
Bobs loss: tensor(0.0044) Alice loss: tensor(0.0282)
Bobs loss: tensor(0.0044) Alice loss: tensor(0.0281)
Bobs loss: tensor(0.0044) Alice loss: tensor(0.0279)
Bobs loss: tensor(0.0044) Alice loss: tensor(0.0277)
Bobs loss: tensor(0.0044) Alice loss: tensor(0.0275)
Bobs loss: tensor(0.0044) Alice loss: tensor(0.0274)
Bobs loss: tensor(0.0043) Alice loss: tensor(0.0274)
Bobs loss: tensor(0.0042) Alice loss: tensor(0.0272)
Bobs loss: tensor(0.0042) Alice loss: tensor(0.0272)
Bobs loss: tensor(0.0041) Alice loss: tensor(0.0271)
Bobs loss: tensor(0.0041) Alice loss: tensor(0.0269)
Bobs loss: tensor(0.0041) Alice loss: tensor(0.0266)
Bobs loss: tensor(0.0041) Alice loss: tensor(0.0265)
Bobs loss: tensor(0.0042) Alice loss: tensor(0.0262)
Bobs loss: tensor(0.0041) Alice loss: tensor(0.0261)
Bobs loss: tensor(0.0040) Alice loss: tensor(0.0261)
Bobs loss: tensor(0.0040) Alice loss: tensor(0

Bobs loss: tensor(0.0024) Alice loss: tensor(0.0158)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0159)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0157)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0157)
Bobs loss: tensor(0.0024) Alice loss: tensor(0.0156)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0156)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0155)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0154)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0154)
Bobs loss: tensor(0.0024) Alice loss: tensor(0.0153)
Bobs loss: tensor(0.0024) Alice loss: tensor(0.0153)
Bobs loss: tensor(0.0024) Alice loss: tensor(0.0152)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0153)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0152)
Bobs loss: tensor(0.0024) Alice loss: tensor(0.0150)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0151)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0150)
Bobs loss: tensor(0.0023) Alice loss: tensor(0.0149)
Bobs loss: tensor(0.0024) Alice loss: tensor(0

Bobs loss: tensor(0.0018) Alice loss: tensor(0.0111)
Bobs loss: tensor(0.0018) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0018) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0111)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0111)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0109)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0.0110)
Bobs loss: tensor(0.0017) Alice loss: tensor(0

Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0015) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0015) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0015) Alice loss: tensor(0.0088)
Bobs loss: tensor(0.0015) Alice loss: tensor(0.0088)
Bobs loss: tensor(0.0015) Alice loss: tensor(0.0088)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0090)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0014) Alice loss: tensor(0.0089)
Bobs loss: tensor(0.0015) Alice loss: tensor(0

...

Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0

Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0

Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0

Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0

Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0.0059)
Bobs loss: tensor(0.0009) Alice loss: tensor(0

In [ ]:
alice_model.weight.data.get()